## **심화 RAG 세션의 목표**

1. LLM을 활용하여 **메타데이터**를 추출하는 방식을 배운다.
2. 메타데이터를 포함하여 **Indexing**을 한다.
2. **Query Summarization**을 통해 대화 맥락이 담긴 기준 쿼리를 생성해낸다.

# **이전 세션 Wrap up**

<img src='https://drive.google.com/uc?export=view&id=1a6pep2MGLAEh5YqPMMRabXd60DYcGlVg' width='800'/>

- Indexing 파트에서는 LLM-Based Metadata Extraction
- Retrieving 파트에서는 Query Summarization

\

# **1. LLM을 활용한 메타데이터 추출** (30분)

## **언제 사용될 수 있는지 예시**

우리가 보유하고 있는 Raw Data와 유저가 물어보는 Query 사이에 간극이 존재할 때!

### **1. 안암 맛집 챗봇**

- 유저가 물어보는 질문
  - 식당의 이름이나 메뉴를 명확히 알고 물어보는 경우는 드물다.
  - 특정한 상황, 원하는 느낌/분위기, 동행인, 리뷰 등을 전반적으로 물어보고 원하는 식당을 찾아낸다.
  - "안암에서 유명한 중국 음식점을 알려줘" -> 간단한 질문
  - "이번에 학회원들이랑 같이 회식을 하려고 하는데, 조금 조용하게 얘기나눌 수 있는 술집을 추천해줘. 안주는 퓨전 요리가 좋을거 같아." -> 복잡한 상황과 요구사항
- 우리가 가진 데이터
  - 식당의 이름, 장소, 메뉴, 리뷰 등
  - 위의 데이터는 유저가 물어보는 질문 형식과 유사도가 많이 떨어진다.
  - 우리가 가진 데이터를 기반으로 **"해당 식당을 어떤 상황에서 가면 좋을지", "누가 가면 좋을지"** 등의 데이터를 추출한다.
  - 이를 사람이 직접 할 수도 있지만, LLM을 활용하며 빠르고 저렴하게 원하는 데이터를 만들 수 있다.
  - "안암동의 대나무 정원은 분위기 있는 인테리어와 저렴하고 맛있는 안주를 가지고 있다. 방문자들은 ㅇㅇㅇ 안주에 대해 높은 평을 했고, 조용하게 이야기를 나누기 좋은 공간이라고 평했다."
  


### **2. 학교 시간표 마법사 챗봇**

- 유저가 물어보는 질문
  - 정확하게 강의명이나 교수님을 물어보는 경우도 있겠지만, 나의 상황, 원하는 수업 방식 등 전반적인 질문을 하는 경우가 많다.
  - "ㅇㅇㅇ 교수님의 수강평이 어때?" -> 특정한 질문 (이거는 그냥 일반 검색 서비스로 충분)
  - "내가 다음 학기에 마케팅 인턴을 하려고 하는데, 실전 프로젝트 위주로 진행되는 마케팅 수업을 듣고 싶어. 교수님도 완전 학계에 있었던 분보다 실제로 산업현장에 있었던 분이면 좋을거 같아." -> 복잡한 상황과 요구사항
- 우리가 가진 데이터
  - 교수님 이력, 수업 계획서, 학생들의 수강평 등
  - 위의 데이터를 각각 chunking 해서 indexing 하는 것보다 이를 기반으로 수업에 대한 설명 텍스트를 인덱싱하는 것이 좋다.
  - 우리가 가진 데이터를 기반으로 **"이 수업은 어떤 상황의 학생이 들으면 좋을지", "어떤 방식으로 진행되며 무엇을 얻어갈 수 있는지"** 등의 데이터를 추출한다.
  - 이를 사람이 직접 할 수도 있지만, LLM을 활용하며 빠르고 저렴하게 원하는 데이터를 만들 수 있다.
  - "ㅇㅇㅇ 교수님의 마케팅 원론은 실전 마케팅 프로젝트 위주로 수업이 진행된다. 산업현장에서 그로스해킹과 관련된 업무를 하셨던 교수님의 경험을 바탕으로 실질적인 피드백과 프로젝트 방향성을 제시한다."
  


### **3. 책 추천 챗봇**

- 유저가 물어보는 질문
  - 정확하게 책이름이나 저자를 물어보는 경우도 있겠지만, 현재 나의 상황, 얻고자 하는 것, 현재 고민 등을 포함한 전반적인 질문을 하는 경우가 많다.
  - "ㅇㅇㅇ 박사님의 심리학 책을 추천해줘" -> 특정한 질문 (이거는 그냥 일반 검색 서비스로 충분)
  - "내가 사업을 운영하고 있는데 매출이 잘 안올라, 마케팅적으로 돌파구를 찾아보려고 하는데 마케팅에 대해서 잘 몰라. 마케팅의 전반적인 입문서로 좋은 책이 있을까?" -> 복잡한 상황과 요구사항
- 우리가 가진 데이터
  - 책 제목, 저자, 목차, 설명, 서평 등
  - 위의 데이터를 각각 chunking 해서 indexing 하는 것보다 이를 기반으로 책에 대한 설명 텍스트를 인덱싱하는 것이 좋다.
  - 우리가 가진 데이터를 기반으로 **"이 책이 어떤 사람에게 도움이 될지", "어떤 도움을 줄 수 있는지"** 등의 데이터를 추출한다.
  - 이를 사람이 직접 할 수도 있지만, LLM을 활용하며 빠르고 저렴하게 원하는 데이터를 만들 수 있다.
  - "세스 고딘의 마케팅이다는 마케팅에 대한 가장 핵심적이고 중요한 관점에 대해서 소개한다. 고객과 문제 중심 사고를 자신이 경험한 다양한 사례를 바탕으로 소개한다."
  


## **슈카월드 영상 한편씩 Summarization Extraction**

file-based metadata extraction에서 추출한 title만 포함하는 경우


```
[title]내가 반도체 산업을 재편하겠다..
[chunk]두 번째 주제는 약간 재미로만 보세요 이게 글로벌 어그로를 끌고 있는 주제가 하나 있습니다 그게 바로 폭주하는 샘 알트먼이라는 주제입니다 개인적으로는 2024년 슈카월드를 책임져줄 수 있는 인물이다 그동안 너무 일론 머스크 아저씨 얘기를 많이 했더니 그 아저씨가 어그로가 세잖아요 관종 그 자체여서 너는 머스크 렉카 아니냐 그래서 바꾸도록 하겠습니다 말하는 거 보니까 심상치 않은 포텐셜을 가지고 계셔요 인방계에는 아주 훌륭한 탤런트가 있다 기대하는 분인데 폭주하는 샘 알트먼 샘 알트먼이 누구냐? 혹시 모를까봐 오픈 AI라는 회사의 챗 지피티 PT 이건 뭔지 아시죠? 여러분들 흔히 쓰는 생성형 AI를 만든 샘 알트먼이라고 할 수 있고요 보면 설날 직전에 원래는 다른 자료를 준비하고 있었는데 월스트리트 저널에 제가 보기에 정말 황당한 기사가 떴어요 샘 알트먼이 AI 반도체 비즈니스를 위해서 수조 달러의 투자를 모으고 있다 이게 월스트릿 저널에
```



summary가 추출된 방식
```
[title]내가 반도체 산업을 재편하겠다..
[summary]주제는 폭주하는 샘 알트먼과 그의 AI 기술에 대한 이야기이며, 샘 알트먼이 오픈 AI를 창업한 인물로서 미래의 기대주로 손꼽히고 있다는 내용을 다루고 있습니다.
[chunk]두 번째 주제는 약간 재미로만 보세요 이게 글로벌 어그로를 끌고 있는 주제가 하나 있습니다 그게 바로 폭주하는 샘 알트먼이라는 주제입니다 개인적으로는 2024년 슈카월드를 책임져줄 수 있는 인물이다 그동안 너무 일론 머스크 아저씨 얘기를 많이 했더니 그 아저씨가 어그로가 세잖아요 관종 그 자체여서 너는 머스크 렉카 아니냐 그래서 바꾸도록 하겠습니다 말하는 거 보니까 심상치 않은 포텐셜을 가지고 계셔요 인방계에는 아주 훌륭한 탤런트가 있다 기대하는 분인데 폭주하는 샘 알트먼 샘 알트먼이 누구냐? 혹시 모를까봐 오픈 AI라는 회사의 챗 지피티 PT 이건 뭔지 아시죠? 여러분들 흔히 쓰는 생성형 AI를 만든 샘 알트먼이라고 할 수 있고요 보면 설날 직전에 원래는 다른 자료를 준비하고 있었는데 월스트리트 저널에 제가 보기에 정말 황당한 기사가 떴어요 샘 알트먼이 AI 반도체 비즈니스를 위해서 수조 달러의 투자를 모으고 있다 이게 월스트릿 저널에
```



### **LLM으로 추출하는 코드**

영상 하나만 해보기

In [ ]:
!pip install langchain
!pip install openai
!pip install tqdm
!pip install pinecone-client
!pip install tiktoken

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid
from openai import OpenAI
from tqdm import tqdm
from pinecone import Pinecone

In [ ]:
OPENAI_API_KEY = ""
PINECONE_API_KEY = ""

In [ ]:
# LLM을 활용하여 summarization 하기
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("data")

client = OpenAI(api_key=OPENAI_API_KEY)

title = "영국인들에게 차를 금지하면 일어나는 일"
with open("/content/영국인들에게 차를 금지하면 일어나는 일.txt") as f:
  text = f.read()


response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {
            "role": "system",
            "content": "you are youtube summarization machine. summarize given youtube script in long paragraph. summarization should be include key points of youtube video. This is economic youtube channel."
        },
        {
            "role": "user",
            "content": f"[title]{title}\n[script]{text}"
        }
    ],
    max_tokens=1024
)

summarization_metadata = response.choices[0].message.content

In [ ]:
summarization_metadata

'이번 영상은 영국의 자존심을 건드리는 것이 어떤 결과를 낳는지에 대해 이야기하고 있습니다. 영국인들은 차를 매우 소중히 여기며 차를 마시는 것이 자부심이라고 언급하며, 영국에서 차를 금지하는 경우에 대해 이야기하고 있습니다. 또한, 미국과 영국이 후티 반군에 대한 공습에 대해 협력하는 과정에서 영국이 활약하는 내용도 다루고 있습니다. 그러나, 최근 후티 반군에 의해 홍해가 막히면서 영국으로의 차 공급에 영향을 주는 상황이 발생하고 있는데, 이에 따라 차의 공급 부족이 우려되고 있습니다. 영국에서는 차 소비량이 미국과 비슷하게 줄어든 것을 언급하며, 미국인들이 차보다 커피를 더 많이 마시는 이유 등에 대해서도 논의하고 있습니다. 영국인들은 차 제조에 대한 정통성과 전통을 중시하는 만큼, 소금을 차에 넣는 것에 대한 논란이 생겼을 때 크나큰 분노를 일으키고 있습니다. 이러한 이야기를 통해 영국과 미국의 차 문화와 관련된 흥미로운 사실들을 알려주고 있습니다. 영상 속에서는 현대적인 소금을 차에 넣는 논란이 영국인들을 어떻게 분노시키는지에 대해 다루고 있습니다. 또한 영국과 미국의 차 문화와 관련된 유머러스한 이야기도 소개하고 있습니다. 전반적으로 영상은 차에 대한 전통과 자부심, 그리고 논란을 통해 영국과 미국의 차 문화에 대해 알려주고 있습니다.'

In [ ]:
# chunk 만들기
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512
)
chunks = text_splitter.split_text(text)

In [ ]:
vectors = []

for chunk in tqdm(chunks, desc="Processing chunks"):
    chunk_with_summarization = f"[title]{title}\n[summarization]{summarization_metadata}\n[script]{chunk}"
    response = client.embeddings.create(input=chunk_with_summarization, model="text-embedding-3-small")
    chunk_embeddings = response.data[0].embedding

    chunk_id = str(uuid.uuid4())

    vector_dict = {
        "id": chunk_id,
        "values": chunk_embeddings,
        "metadata": {"chunk": chunk_with_summarization}
    }
    vectors.append(vector_dict)

batch_size = 10
num_batches = len(vectors) // 10 + 1

for i in tqdm(range(num_batches), desc="Processing chunks"):
    start_idx = i * batch_size
    end_idx = start_idx + batch_size

    batch_vectors = vectors[start_idx:end_idx]

    # index.upsert(vectors=batch_vectors, namespace="ns3")

Processing chunks: 100%|██████████| 10/10 [00:08<00:00,  1.18it/s]


\

## **슈카월드 영상 Question Extraction**

[파일 다운로드 링크](https://drive.google.com/file/d/1QWl0m0thSXKpaubbaS05ZmeCmCpMAery/view?usp=sharing)

## **prompt engineering → 각자 실습 하기**

- 주어진 chunk 일부를 불러와서 playground나 colab에서 prompt를 작성하여 원하는 값 나오게 해보기
- 최적의 prompt 찾기
- 자르는 chunk의 크기 정하기

[playground 링크](https://platform.openai.com/playground/chat)

**< 출력해야 하는 것>**

1. chunk의 요약
2. 예상 질문

**<예시>**


```
[summary]
주제는 폭주하는 샘 알트먼과 그의 AI 기술에 대한 이야기이며,
샘 알트먼이 오픈 AI를 창업한 인물로서 미래의 기대주로 손꼽히고 있다는 내용을 다루고 있습니다.

[questions]
1. 샘 알트먼은 누구인가요?
2. 샘 알트먼이 폭주하는 이유는 무엇인가요?
3. 샘 알트먼이 AI 반도체 비즈니스를 위해 수조 달러의 투자를 모으고 있다는데, 이에 대해 어떤 반응이 있나요?

[chunk]
두 번째 주제는 약간 재미로만 보세요 이게 글로벌 어그로를 끌고 있는 주제가 하나 있습니다
그게 바로 폭주하는 샘 알트먼이라는 주제입니다 개인적으로는 2024년 슈카월드를 책임져줄 수 있는 인물이다
.
.
.
날 직전에 원래는 다른 자료를 준비하고 있었는데 월스트리트 저널에 제가 보기에 정말 황당한 기사가 떴어요
샘 알트먼이 AI 반도체 비즈니스를 위해서 수조 달러의 투자를 모으고 있다 이게 월스트릿 저널에
```



In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open("/content/영국인들에게 차를 금지하면 일어나는 일.txt") as f:
    text = f.read()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
      chunk_size=1024
  )
texts = text_splitter.split_text(text)

In [ ]:
texts[0]

'두 번째 주제는 약간 재미로만 보세요 이게 글로벌 어그로를 끌고 있는 주제가 하나 있습니다 그게 바로 폭주하는 샘 알트먼이라는 주제입니다 개인적으로는 2024년 슈카월드를 책임져줄 수 있는 인물이다 그동안 너무 일론 머스크 아저씨 얘기를 많이 했더니 그 아저씨가 어그로가 세잖아요 관종 그 자체여서 너는 머스크 렉카 아니냐 그래서 바꾸도록 하겠습니다 말하는 거 보니까 심상치 않은 포텐셜을 가지고 계셔요 인방계에는 아주 훌륭한 탤런트가 있다 기대하는 분인데 폭주하는 샘 알트먼 샘 알트먼이 누구냐? 혹시 모를까봐 오픈 AI라는 회사의 챗 지피티 PT 이건 뭔지 아시죠? 여러분들 흔히 쓰는 생성형 AI를 만든 샘 알트먼이라고 할 수 있고요 보면 설날 직전에 원래는 다른 자료를 준비하고 있었는데 월스트리트 저널에 제가 보기에 정말 황당한 기사가 떴어요 샘 알트먼이 AI 반도체 비즈니스를 위해서 수조 달러의 투자를 모으고 있다 이게 월스트릿 저널에'

In [ ]:
#openai api key 입력하기

OPENAI_API_KEY = "api key 입력"

In [ ]:
#system prompt와 query를 입력하면 답변 출력해주는 함수
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

def chat_completions(system, query, model = "gpt-3.5-turbo"):
  messages = [
      {
        "role": "system",
        "content": system
    },
    {
        "role":"user",
        "content":query
    }
  ]
  response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.5,
        max_tokens=512,
    )
  result = response.choices[0].message.content

  return result

In [ ]:
#요약내용을 출력하는 프롬프트 예시
system_summary = "입력할 내용은 유튜브 영상 스크립트야. 내용을 한 문단으로 요약해줘."

In [ ]:
#예상 질문 출력하는 프롬프트 예시
system_question = "사용자가 질문할 만한 예상 질문들을 입력한 내용과 관련해서 3개를 뽑아줘"

In [ ]:
#프롬프트 답변 확인하기
response = chat_completions(system_summary, texts[0])
response

'두 번째 주제는 폭주하는 샘 알트먼에 대한 이야기이다. 샘 알트먼은 오픈 AI의 창업자로, 생성형 AI 기술을 개발하는 인물이다. 최근에는 AI 반도체 비즈니스를 위해 수조 달러의 투자를 모으고 있다고 한다.'

In [ ]:
#프롬프트 답변 확인하기
response = chat_completions(system_question, texts[0])
response

'1. 샘 알트먼은 누구인가요?\n2. 샘 알트먼이 만든 생성형 AI인 오픈 AI의 챗 지피티 PT는 무엇인가요?\n3. 샘 알트먼이 AI 반도체 비즈니스를 위해 수조 달러의 투자를 모으고 있다는데, 이에 대한 자세한 내용이 어디에 실렸는지 궁금합니다.'

\

### **미리 Extracting을 수행한 파일로 indexing 수행하기**

[파일 다운로드 링크](https://drive.google.com/file/d/1hyhK0IlGuVAVTfF9A1ys6ZH8hq862Q1I/view?usp=sharing)

In [ ]:
# 파일을 열고 내용을 읽은 후 '--------'로 분할
chunks_indexing = []
with open('/content/indexing_summary_question.txt', 'r', encoding='utf-8') as file:
    content = file.read()
    chunks_indexing = content.split('--------')

In [ ]:
#chunking한 것을 임베딩한 후 vector DB에 넣기

vectors = []

for chunk in tqdm(chunks_indexing, desc="Processing chunks"):
    response = client.embeddings.create(input= chunk, model="text-embedding-3-small")
    chunk_embeddings = response.data[0].embedding

    chunk_id = str(uuid.uuid4())

    vector_dict = {
        "id": chunk_id,
        "values": chunk_embeddings,
        "metadata": {"chunk": chunk}
    }
    vectors.append(vector_dict)

batch_size = 10
num_batches = len(vectors) // 10 + 1

for i in tqdm(range(num_batches), desc="Processing chunks"):
    start_idx = i * batch_size
    end_idx = start_idx + batch_size

    batch_vectors = vectors[start_idx:end_idx]

    index.upsert(vectors=batch_vectors, namespace="ns3")

\

# **2. [RAG 심화]Query Summarization 하기** (30분)

- 기존의 RAG 방식의 문제는 유저가 방금 입력한 쿼리를 항상 기준 임베딩(벡터)로 만들어서 가져오게 된다.
- 만약에 "이번에 레딧이 상장하면서 가장 수혜를 본 사람이 누구야?"라고 물어봤을 때는 임베딩을 통해서 비슷한 의미를 가진 chunk를 가져올 것이다.
- 근데 이어서 "그럼 왜 많은 수혜를 봤어?" 라고 물어보면 이 쿼리를 기준 임베딩(벡터)로 만들게 되고, 그러면 위의 맥락은 없는 "그럼 왜 많은 수혜를 봤어?"와 유사한 의미의 chunk를 가져오게 되서 원하는 결과를 얻지 못한다.
- 그래서 이전 쿼리와 맥락들을 담고 있는 요약본을 만들고 이를 기준 임베딩으로 만들어서 chunk를 가져온다.

<img src='https://drive.google.com/uc?export=view&id=1TdTF1TL-PCeY0Lrflk78qIhdHw-040y7' width='600'/>

<img src='https://drive.google.com/uc?export=view&id=1_yD6UmNBtdV5bEH8WhwnAg7O3CvoQ4M7' width='600'/>

<img src='https://drive.google.com/uc?export=view&id=19jQNiM1gs2pOPUEkAexlvJMyDem698u0' width='600'/>

### **방식 1. 모든 이전 유저의 입력을 활용하여 요약하기**

<img src='https://drive.google.com/uc?export=view&id=1YdIYP50GOAmC1MlA4y9A9Qt5QYPcL-A7' width='600'/>

In [ ]:
messages = [
    {
        "role": "user",
        "content": "레딧의 상장으로 가장 수혜를 본 사람은 누구야?"
    },
    {
        "role": "assistant",
        "content": """레딧의 상장으로 가장 수혜를 본 사람은 샘 알트먼입니다.
        샘 알트먼은 레딧에 10년 전부터 투자를 해왔고, 레딧의 3대 주주 중 개인 최대 투자자로 알려져 있습니다.
        그의 지분 가치는 6억 달러에서 7800억 달러까지 증가했으며, 앞으로 어떻게 될지는 확실하지 않지만 AI로도
        레딧에서 수익을 창출할 수 있는 조만 장자로 등극할 것으로 예상됩니다. 샘 알트먼은 레딧 상장으로 큰 이익을 얻은 것으로 알려져 있습니다. 🚀"""
    },
    {
        "role": "user",
        "content": "그럼 다음은?"
    },
    {
        "role": "assistant",
        "content": "잘 모르겠습니다."
    }
]

[playground 링크](https://platform.openai.com/playground/chat)

플레이그라운드에서 메시지 입력하고 요약하는 시스템 프롬프트 찾기!

In [ ]:
recent_user_query = ""
for message in messages:
  if message["role"] == "user":
    recent_user_query += f'[{message["role"]}]\n{message["content"]}\n]n'

recent_query_prompt = {
    "role": "user",
    "content": f"지금까지 유저가 입력한 질문입니다.\n{recent_user_query}"
}

summarization_system_prompt = {
    "role": "system",
    "content": "유저가 입력한 여러 질문을 파악하고 하나의 긴 질문으로 변환해줘."
}


response = client.chat.completions.create(
  # model="gpt-4-0125-preview",
  model="gpt-3.5-turbo-0125",
  messages=[summarization_system_prompt, recent_query_prompt],
  temperature=0.5,
  max_tokens=1024,
)

summarized_queries = response.choices[0].message.content

In [ ]:
summarized_queries

'지금까지 유저가 입력한 질문은 "레딧의 상장으로 가장 수혜를 본 사람은 누구야? 그럼 다음은?"입니다.'

\

### **방식 2. 방금 유저 쿼리와 최신 질문-답변 2쌍을 사용**

<img src='https://drive.google.com/uc?export=view&id=1KaKt_BWSaPJlgdSuD88MiOtSYFaCqF1y' width='600'/>

In [ ]:
recent_query = ""
if len(messages) >= 2:
  for message in messages[-3:]:
    recent_query += message["role"] + "\n" + message["content"] + "\n\n"
else:
  for message in messages:
    recent_query += message["content"] + "\n"

recent_query_prompt = {
    "role": "user",
    "content": f"지금까지 주고받은 메시지\n{recent_query}"
}

summarization_system_prompt = {
    "role": "system",
    "content": "여기 채우기"
}


response = client.chat.completions.create(
  model=st.session_state.openai_model,
  messages=[summarization_system_prompt, recent_query_prompt],
  temperature=0.5,
  max_tokens=1024,
)

summarized_queries = response.choices[0].message.content

\

## **슈카월드 챗봇에 구현하기**

In [ ]:
!pip install streamlit
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00


In [ ]:
!ngrok authtoken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile app.py
print("hello")

Writing app.py


In [ ]:
from pyngrok import ngrok
!streamlit run app.py&>/dev/null&
publ_url = ngrok.connect(addr='8501')
publ_url

<NgrokTunnel: "https://5dde-34-83-24-245.ngrok-free.app" -> "http://localhost:8501">

In [ ]:
# app.py 에서 직접 수정하겠습니다.
import streamlit as st
import uuid
from openai import OpenAI
from tqdm import tqdm
from pinecone import Pinecone

OPENAI_API_KEY = ""
PINECONE_API_KEY = ""

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("data")

client = OpenAI(api_key=OPENAI_API_KEY)

st.set_page_config(layout="wide")

st.header("🐯슈카월드 챗봇🐯")
if "openai_model" not in st.session_state:
  st.session_state.openai_model = "gpt-3.5-turbo-0125"

if "messages" not in st.session_state:
  system_prompt = {
      "role": "system",
      "content": """당신은 경제/시사를 다루는 유튜브 채널인 슈카월드의 정보를 알려주는 챗봇입니다.
      슈카월드를 당신보다 많이 알고 있는 존재는 이 우주에 없습니다.
      경제/시사 이슈를 물어보는 사람들에게 슈카월드에서 다룬 내용을 바탕으로 설명해주고 질문에 답을 해주는 역할을 맡고 있습니다.
      당신은 사람들에게 다음과 같이 대답해야 합니다:
      - 친절한 말투
      - 항상 존댓말 사용
      - 적절한 이모지 사용
      당신은 반드시 제공하는 [Context]에 있는 내용을 기반으로 살을 붙여 답변을 생성해야 합니다."""
  }
  st.session_state.messages = [system_prompt]

if "retrievied_chunks" not in st.session_state:
  st.session_state.retrievied_chunks = []

if "summarized_query" not in st.session_state:
  st.session_state.summarized_query = ""

for message in st.session_state.messages:
  if message["role"] != "system":
    with st.chat_message(message["role"]):
      st.markdown(message["content"])

with st.container(height=650):
  col1, col2 = st.columns([5,5], gap="medium")
  with col1:
    st.subheader("Chat Section")
    if prompt := st.chat_input("슈카월드에서 궁금한 정보를 물어보세요😊"):
      user_prompt = {
        "role": "user",
        "content": prompt
      }
      st.session_state.messages.append(user_prompt)

      with st.chat_message("user"):
        st.markdown(prompt)

      with st.chat_message("assistant"):
        # Query Summarization 수행하기
        recent_user_query = ""
        # for message in st.session_state.messages:
        #   if message["role"] == "user":
        #     recent_user_query += message["role"] + "\n" + message["content"] + "\n\n"

        # recent_query_prompt = {
        #     "role": "user",
        #     "content": f"지금까지 유저가 입력한 질문입니다.\n{recent_user_query}"
        # }

        # summarization_system_prompt = {
        #     "role": "system",
        #     "content": "여기 채우기"
        # }

        # response = client.chat.completions.create(
        #   model=st.session_state.openai_model,
        #   messages=[summarization_system_prompt, recent_query_prompt],
        #   temperature=0.5,
        #   max_tokens=1024,
        # )

        # summarized_query = response.choices[0].message.content

        #요약한 query를 embeddings로 변경
        response = client.embeddings.create(input=prompt, model="text-embedding-3-small")
        query_embeddings = response.data[0].embedding

        # context 가져오기
        retrieved_chunks = index.query(
            namespace="ns3",
            vector=query_embeddings,
            top_k=7,
            include_values=False,
            include_metadata=True,
        )
        contexts = ""

        for match in retrieved_chunks.matches:
            context = match["metadata"]["chunk"]
            contexts += context + "\n\n"
            st.session_state.retrievied_chunks.append(context)

        context_prompt = {
            "role" : "system",
            "content" : f"[Context]\n{contexts}"
        }

        # 전체 활용하여 LLM 호출
        messages = st.session_state.messages
        messages.append(context_prompt)

        message_placeholder = st.empty()
        full_response = ""

        for response in client.chat.completions.create(
              model=st.session_state.openai_model,
              messages=messages,
              temperature=0.5,
              max_tokens=1024,
              stream=True
        ):
          full_response += (response.choices[0].delta.content or "")
          message_placeholder.markdown(full_response + "▌")
        message_placeholder.markdown(full_response)
      st.session_state.messages.append({"role": "assistant", "content": full_response})

    with col2:
      st.subheader("Logging Section")
      st.write("Summarized Query")
      st.write(st.session_state.summarized_query)
      st.write("Retrieved Chunks")
      for idx, chunk in enumerate(st.session_state.retrievied_chunks):
        st.write(f"{idx+1}th chunk: {chunk}")

In [ ]:
!ps

In [ ]:
!kill

# **실습 과제: 교보문고** (30분)

아래와 같이 원하는 Indexing 형태를 만들어보세요. LLM을 활용하여 데이터를 추출하고 텍스트를 구성하세요.

```
[title]책제목\n[적합한 상황]책이 도움이 될 만한 상황\n[책설명]책에 대한 요약설명
```



[책 데이터](https://drive.google.com/file/d/1XkFNdy6_xkdAnsZCuN-X2zG9cp__lfqE/view?usp=sharing)

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

df = pd.read_csv('경로 복사해서 넣기')
datas = df.iterrows()

# 첫번째 데이터만 가지고 테스트해보기
test_data = datas[0]
title = test_data['제목']
title_link = test_data['제목_링크']
item_info = test_data['item_info']
rating_rvCount = test_data['rating_rvCount']
yes_b = test_data['yes_b']
detailed_title = test_data['상세 제목']
book_intro = test_data['책소개']
table_of_contents = test_data['목차']
in_book = test_data['책속으로']
publisher_review = test_data['출판사 리뷰']

# 먼저 LLM을 활용하여 책에 대한 요약설명을 뽑아내기

# 다음으로 책이 도움이 될 만한 상황을 뽑아내기

# 텍스트를 합쳐서 Embeddings 만들기

# for문으로 구성하여 전체 ns4에 indexing 구성하기
